## Carregando e Explorando o Conjunto de Dados

In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Create a SparkConf object
conf = SparkConf().setAppName("ProductDataAnalysis") \
           .setMaster("local[2]") \
           .set("spark.executor.memory", "1g")

# Create a SparkSession object
spark = SparkSession.builder.config(conf=conf).getOrCreate()

# Retrieve the SparkConf object from the SparkContext
conf = spark.sparkContext.getConf()

# Print the configuration settings
print("spark.app.name = ", conf.get("spark.app.name"))
print("spark.master = ", conf.get("spark.master"))
print("spark.executor.memory = ", conf.get("spark.executor.memory"))

spark.app.name =  ProductDataAnalysis
spark.master =  local[2]
spark.executor.memory =  1g


In [2]:
# Carregue os dados do arquivo CSV que já está dentro do seu projeto
file_path = "product+classification+and+clustering/pricerunner_aggregate.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [3]:
df.show(5)

+----------+--------------------+------------+-----------+--------------------+------------+---------------+
|Product ID|       Product Title| Merchant ID| Cluster ID|       Cluster Label| Category ID| Category Label|
+----------+--------------------+------------+-----------+--------------------+------------+---------------+
|         1|apple iphone 8 pl...|           1|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         2|apple iphone 8 pl...|           2|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         3|apple mq8n2b/a ip...|           3|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         4|apple iphone 8 pl...|           4|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
|         5|apple iphone 8 pl...|           5|          1|Apple iPhone 8 Pl...|        2612|  Mobile Phones|
+----------+--------------------+------------+-----------+--------------------+------------+---------------+
only showing top 5 

In [4]:
df.printSchema()

root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |--  Merchant ID: integer (nullable = true)
 |--  Cluster ID: integer (nullable = true)
 |--  Cluster Label: string (nullable = true)
 |--  Category ID: integer (nullable = true)
 |--  Category Label: string (nullable = true)



In [5]:
df.describe().show()

+-------+-----------------+--------------------+------------------+------------------+--------------------+------------------+----------------+
|summary|       Product ID|       Product Title|       Merchant ID|        Cluster ID|       Cluster Label|       Category ID|  Category Label|
+-------+-----------------+--------------------+------------------+------------------+--------------------+------------------+----------------+
|  count|            35311|               35311|             35311|             35311|               35311|             35311|           35311|
|   mean|26150.80017558268|                null|120.50188326583785|30110.687632749003|                null| 2618.142929965167|            null|
| stddev|13498.19122018199|                null|117.04555721851422|18410.265642128295|                null|3.6007080727956833|            null|
|    min|                1|10 hd portable te...|                 1|                 1|AEG A71101TSX0 St...|              2612|          

In [6]:
print(spark.sparkContext.getConf().getAll())

[('spark.driver.extraJavaOptions', '-XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED'), ('spark.app.submitTime', '1703852316041'), ('spark.app.id', 'local-1703852317194'), ('spark.driver.host', 'f58eefab1b97'), ('spark.executor.id', 'driver'), ('spark.app.startTime', '1703852316193'), ('spark.app.name', 'ProductDataAnalys

## Limpeza e Preparação dos Dados

In [7]:
from pyspark.sql.functions import col
df.select([col(c).isNull().alias(c) for c in df.columns]).show()

+----------+-------------+------------+-----------+--------------+------------+---------------+
|Product ID|Product Title| Merchant ID| Cluster ID| Cluster Label| Category ID| Category Label|
+----------+-------------+------------+-----------+--------------+------------+---------------+
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false|      false|         false|       false|          false|
|     false|        false|       false| 

In [8]:
# Exemplo de exclusão de linhas com valores nulos
df_clean = df.na.drop()

In [9]:
df.printSchema()

root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |--  Merchant ID: integer (nullable = true)
 |--  Cluster ID: integer (nullable = true)
 |--  Cluster Label: string (nullable = true)
 |--  Category ID: integer (nullable = true)
 |--  Category Label: string (nullable = true)



In [10]:
# Lista de nomes de colunas conforme o esquema com espaços
col_names = df_clean.schema.names

# Renomear colunas para remover espaços iniciais
for col_name in col_names:
    new_col_name = col_name.strip()  # strip() remove espaços do começo e do fim
    df_clean = df_clean.withColumnRenamed(col_name, new_col_name)

# Verificando o novo esquema
df_clean.printSchema()

root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |-- Merchant ID: integer (nullable = true)
 |-- Cluster ID: integer (nullable = true)
 |-- Cluster Label: string (nullable = true)
 |-- Category ID: integer (nullable = true)
 |-- Category Label: string (nullable = true)



In [11]:
# Exemplo de conversão de uma coluna de string para inteiro
df_clean = df_clean.withColumn("Merchant ID", df_clean["Merchant ID"].cast("integer"))

In [12]:
# NOTA: A conversão de tipos de dados é útil quando você precisa alterar o tipo de uma coluna. 
# No nosso DataFrame, a coluna 'Merchant ID' já é do tipo inteiro, conforme mostrado pelo esquema:
df_clean.printSchema()

# Portanto, a conversão para inteiro não é necessária neste caso. Se fosse uma coluna do tipo string
# que contém apenas números, a conversão seria realizada da seguinte maneira:
# df_clean = df_clean.withColumn("Merchant ID", df_clean["Merchant ID"].cast("integer"))

root
 |-- Product ID: integer (nullable = true)
 |-- Product Title: string (nullable = true)
 |-- Merchant ID: integer (nullable = true)
 |-- Cluster ID: integer (nullable = true)
 |-- Cluster Label: string (nullable = true)
 |-- Category ID: integer (nullable = true)
 |-- Category Label: string (nullable = true)



## Análise Exploratória de Dados

In [13]:
#Calculando a Distribuição de Produtos por Categoria
from pyspark.sql.functions import count

# Agrupando por categoria e contando os produtos
categoria_distribuicao = df_clean.groupBy("Category Label").agg(count("Product ID").alias("Count")).orderBy("Count", ascending=False)

# Visualizando o resultado
categoria_distribuicao.show()

+----------------+-----+
|  Category Label|Count|
+----------------+-----+
| Fridge Freezers| 5501|
|   Mobile Phones| 4081|
|Washing Machines| 4044|
|            CPUs| 3862|
|         Fridges| 3584|
|             TVs| 3564|
|     Dishwashers| 3424|
| Digital Cameras| 2697|
|      Microwaves| 2342|
|        Freezers| 2212|
+----------------+-----+



In [14]:
# Identificando os Comerciantes com Mais Ofertas
comerciantes_top = df_clean.groupBy("Merchant ID").agg(count("Product ID").alias("Total Products")).orderBy("Total Products", ascending=False)

comerciantes_top.show()

+-----------+--------------+
|Merchant ID|Total Products|
+-----------+--------------+
|          3|          2547|
|          6|          1591|
|        298|          1523|
|         31|          1350|
|        119|          1239|
|          7|          1204|
|         17|          1193|
|        293|          1177|
|        294|          1000|
|        301|           901|
|         22|           860|
|        131|           760|
|        300|           736|
|         14|           699|
|         48|           663|
|         15|           661|
|        125|           642|
|         98|           593|
|        128|           583|
|         64|           582|
+-----------+--------------+
only showing top 20 rows



In [15]:
# Importando a função necessária
from pyspark.sql.functions import countDistinct

# Contando a quantidade de títulos de produtos únicos em cada categoria
diversidade_categoria = df_clean.groupBy("Category Label").agg(countDistinct("Product Title").alias("Unique Product Titles"))

# Exibindo o resultado
diversidade_categoria.show()

+----------------+---------------------+
|  Category Label|Unique Product Titles|
+----------------+---------------------+
|      Microwaves|                 2107|
|         Fridges|                 3211|
|             TVs|                 3295|
|Washing Machines|                 3426|
|        Freezers|                 1911|
| Digital Cameras|                 2411|
|            CPUs|                 3062|
|     Dishwashers|                 3060|
| Fridge Freezers|                 4830|
|   Mobile Phones|                 3682|
+----------------+---------------------+



In [1]:
from pyspark import SparkContext

# Iniciando o SparkContext
sc = SparkContext("local", "MapPartitionsExample")

# Exemplo de dados de vendas (ID da venda, Valor da venda, Data da venda, ID do vendedor)
data = [(1, 100, '2021-01-01', 'A1'), (2, 200, '2021-01-02', 'A2'), 
        (3, 300, '2021-01-03', 'A3'), (4, 400, '2021-01-04', 'A4'), 
        (5, 500, '2021-01-05', 'A5')]

sales_rdd = sc.parallelize(data,2)

num_partitions = sales_rdd.getNumPartitions()
print("Número de partições:", num_partitions)

# Função para somar vendas em cada partição
def sum_sales(iterator):
    yield sum(sale_amount[1] for sale_amount in iterator)

# Usando mapPartitions para pré-agregar as vendas em cada partição
partial_sums = sales_rdd.mapPartitions(sum_sales)

# Usando reduce para calcular a soma total
total_sales = partial_sums.reduce(lambda x, y: x + y)

print("Total Sales:", total_sales)

Número de partições: 2
Total Sales: 1500


In [1]:
from pyspark import SparkContext

# Iniciando o SparkContext
sc = SparkContext("local", "PartitionExample")

# Exemplo de dados
data = [(1, 100, '2021-01-01', 'A1'), (2, 200, '2021-01-02', 'A2'), 
        (3, 300, '2021-01-03', 'A3'), (4, 400, '2021-01-04', 'A4'), 
        (5, 500, '2021-01-05', 'A5')]
sales_rdd = sc.parallelize(data,2)

num_partitions = sales_rdd.getNumPartitions()
print("Número de partições:", num_partitions)
print(" ")

# Função para somar vendas em cada partição
def sum_sales(iterator):
    yield sum(sale_amount[1] for sale_amount in iterator)
    
# Função para imprimir os registros em cada partição
def show_records_in_partition(index, iterator):
    yield f"Partição: {index}"
    for record in iterator:
        yield record

# Aplicando a função para visualizar os dados em cada partição
partitioned_records = sales_rdd.mapPartitionsWithIndex(show_records_in_partition)

# Coletando e imprimindo os resultados
for record in partitioned_records.collect():
    print(record)
    
# Usando mapPartitions para pré-agregar as vendas em cada partição
total_sales = sales_rdd.mapPartitions(sum_sales).reduce(lambda x, y: x + y)

print("\nTotal Sales:", total_sales)


Número de partições: 2
 
Partição: 0
(1, 100, '2021-01-01', 'A1')
(2, 200, '2021-01-02', 'A2')
Partição: 1
(3, 300, '2021-01-03', 'A3')
(4, 400, '2021-01-04', 'A4')
(5, 500, '2021-01-05', 'A5')

Total Sales: 1500


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("OptimizationExample") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

# Consultando a configuração
shuffle_partitions = spark.conf.get("spark.sql.shuffle.partitions")
print("Número atual de shuffle partitions:", shuffle_partitions)


Número atual de shuffle partitions: 100


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Inicializando a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("DataOptimizationExample") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

# Carregando os dados
df = spark.read.csv("path/to/large/dataset")

# Transformações complexas
transformed_df = df.select(col("col1"), col("col2")).where(col("col2") > 100)

# Agregação eficiente
result_df = transformed_df.groupBy("col1").count()

# Salvar o resultado
result_df.write.format("parquet").save("path/to/output")
